In [ ]:
from fastai2.text.all import *

path = untar_data(URLs.HUMAN_NUMBERS)

Path.BASE_PATH = path

lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())


text = ' . '.join([l.strip() for l in lines])
tokens = text.split(' ')
tokens = text.split(' ')
vocab = L(*tokens).unique()
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)


##################################
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
bs = 64
cut = int(len(seqs) * 0.8)
#d = default_device()
#print(f"device is {d}") # xm.xla_device()
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False,
                            # device=d,
                             drop_last=True
                            )
dls.device

In [ ]:
from fastai2.text.all import *

class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        print("LMModel5 init")
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        print("LMModel5 end init")
        
    def forward(self, x):
        print("LMModel5 forward start")
        res,h = self.rnn(self.i_h(x), self.h)
        print("LMModel5 forward detaching")
        self.h = h.detach()
        print("LMModel5 forward calculating output")
        ooo = self.h_o(res)
        print("LMModel5 forward returning output")
        return ooo
    
    def reset(self):
      print("LMModel5 init reset")
      self.h.zero_()
      print("LMModel5 end reset")

In [ ]:
%%time
from fastai2.test_utils import VerboseCallback
import pdb

class CountBatches(Callback):
  def begin_fit(self):
    print(f"begin_fit lenghts => {len(self.learn.dls.train)}, {len(self.learn.dls.valid)}")
  def begin_batch(self):
    print(f"b_b={self.learn.iter}")

@patch_to(BaseLoss)
def __call__(self, inp, targ, **kwargs):
        inp  = inp .transpose(self.axis,-1).contiguous()
        targ = targ.transpose(self.axis,-1).contiguous()
        if self.floatify and targ.dtype!=torch.float16: targ = targ.float()
        if targ.dtype in [torch.int8, torch.int16, torch.int32]: targ = targ.long()
        if self.flatten: inp = inp.view(-1,inp.shape[-1]) if self.is_2d else inp.view(-1)
        pdb.set_trace()
        return self.func.__call__(inp, targ.view(-1) if self.flatten else targ, **kwargs)

def loss_func(inp, targ):
    print("loss_func", inp.shape, targ.shape, "loss_func")
    return F.cross_entropy(inp.view(-1, len(vocab)), targ.view(-1))
@log_args(but='cbs')
@patch_to(Learner)
def fit(self, n_epoch, lr=None, wd=None, cbs=None, reset_opt=False):
    print("==================================== add callbacks!!!!")
    with self.added_cbs(cbs):
            if reset_opt or not self.opt: self.create_opt()
            if wd is None: wd = self.wd
            if wd is not None: self.opt.set_hypers(wd=wd)
            self.opt.set_hypers(lr=self.lr if lr is None else lr)

            try:
                self._do_begin_fit(n_epoch)
                for epoch in range(n_epoch):
                    try:
                        print(f"doing epoch {self.epoch} of {len(self.dls.train)} and {len(self.dls.valid)}")
                        self.epoch=epoch;          self('begin_epoch')
                        print("-> self._do_epoch_train()")
                        self._do_epoch_train()
                        print("<- self._do_epoch_train()")
                        print("---> self._do_epoch_validate()")
                        self._do_epoch_validate()
                        print("<--- self._do_epoch_validate()")
                    except CancelEpochException:   self('after_cancel_epoch')
                    finally:                       self('after_epoch')

            except CancelFitException:             self('after_cancel_fit')
            finally:
                print(f"+++++++++++++++++++++++ WILL CALL AFTER FIT {self}")
                self('after_fit')
                print("CALLED!!!! AFTER FIT")
                self._end_cleanup()
                print("************* ###### CALLED!!!! cleanup")
    print("!!!!!!!!!!!!!!! ! ! ! ! !!!!!!!!!!!!! ENDED with self.callbacks....")
    pdb.set_trace()

@patch_to(Learner)
def all_batches(self):
        self.n_iter = len(self.dl)
        print(f"????????????????????????? self.n_iter={self.n_iter}")
        for o in enumerate(self.dl):
          print("??????????????????????????? self.one_batch")
          self.one_batch(*o)
          print("??????????????????????????? self.one_batch did end!!!")
@patch_to(Learner)
def one_batch(self, i, b):
        self.iter = i
        try:
            print("--------------> --------------> --------------> ### 0")
            self._split(b);                                  self('begin_batch')
            print("--------------> --------------> --------------> ### 1")
            self.pred = self.model(*self.xb);                self('after_pred')
            print("--------------> --------------> --------------> ### 2")
            if len(self.yb) == 0: return
            print(f"--------------> --------------> --------------> ### 3 {self.pred.shape} and lenth ={len(self.yb)} yb_0={self.yb[0].shape}")
            self.loss = self.loss_func(self.pred, *self.yb); self('after_loss')
            print("--------------> --------------> --------------> ### 4")
            if not self.training: return
            print("--------------> --------------> --------------> ### 5")
            print("############ --------------> --------------> -------------->")
            self.loss.backward();                            self('after_backward')
            print("############ <-------------- <-------------- <--------------")
            print("--------------> --------------> --------------> ### 6")
            self.opt.step();                                 self('after_step')
            print("--------------> --------------> --------------> ### 7")
            self.opt.zero_grad()
            print("--------------> --------------> --------------> ### 8.0")
        except CancelBatchException:                         self('after_cancel_batch')
        finally:
          print("--------------> --------------> --------------> ### 9999999")
          print("¿¿¿¿¿¿¿¿¿¿¿¿¿¿will call after batch!!!")
          self('after_batch')
          print("¿¿¿¿¿¿¿¿¿¿¿¿¿¿      called after batch!!!")

learn = Learner(dls, LMModel5(len(vocab), 64, 1), 
                #loss_func=CrossEntropyLossFlat(), 
                loss_func=loss_func,
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(1, 3e-3, cbs=[CountBatches(), VerboseCallback()])

# try again

In [ ]:
from fastai2.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

In [ ]:

Path.BASE_PATH = path

In [ ]:
lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
lines

In [ ]:
text = ' . '.join([l.strip() for l in lines])


In [ ]:
tokens = text.split(' ')


In [ ]:
vocab = L(*tokens).unique()


In [ ]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)

In [ ]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))


In [ ]:
#bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

In [ ]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [ ]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

# try again again again

In [2]:
from fastai2.text.all import *
path = untar_data(URLs.HUMAN_NUMBERS)

#hide
Path.BASE_PATH = path

lines = L()
with open(path/'train.txt') as f: lines += L(*f.readlines())
with open(path/'valid.txt') as f: lines += L(*f.readlines())
    
text = ' . '.join([l.strip() for l in lines])

tokens = text.split(' ')

vocab = L(*tokens).unique()


word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)

seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)

If you dont execute this two you get the `ValueError: Expected input batch_size (192) to match target batch_size (64).` So lets execute them

In [5]:
def group_chunks(ds, bs):
    m = len(ds) // bs
    new_ds = L()
    for i in range(m): new_ds += L(ds[i + m*j] for j in range(bs))
    return new_ds

In [6]:
sl = 16
seqs = L((tensor(nums[i:i+sl]), tensor(nums[i+1:i+sl+1]))
         for i in range(0,len(nums)-sl-1,sl))
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(group_chunks(seqs[:cut], bs),
                             group_chunks(seqs[cut:], bs),
                             bs=bs, drop_last=True, shuffle=False)

In [7]:
class LMModel5(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = torch.zeros(n_layers, bs, n_hidden)
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(res)
    
    def reset(self): self.h.zero_()

In [8]:
learn = Learner(dls, LMModel5(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.056531,2.651854,0.461263,00:01
1,2.138902,1.819939,0.468831,00:01
2,1.696971,1.985687,0.324137,00:01
3,1.477333,1.823898,0.450765,00:01
4,1.299323,1.970886,0.482747,00:01
5,1.125414,2.154227,0.499756,00:01
6,0.968370,2.304337,0.518717,00:01
7,0.839025,2.417535,0.521484,00:01
8,0.727712,2.488352,0.530111,00:01
9,0.632702,2.518758,0.532878,00:01


### LSTM 1

In [9]:
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.forget_gate = nn.Linear(ni + nh, nh)
        self.input_gate  = nn.Linear(ni + nh, nh)
        self.cell_gate   = nn.Linear(ni + nh, nh)
        self.output_gate = nn.Linear(ni + nh, nh)

    def forward(self, input, state):
        h,c = state
        h = torch.stack([h, input], dim=1)
        forget = torch.sigmoid(self.forget_gate(h))
        c = c * forget
        inp = torch.sigmoid(self.input_gate(h))
        cell = torch.tanh(self.cell_gate(h))
        c = c + inp * cell
        out = torch.sigmoid(self.output_gate(h))
        h = outgate * torch.tanh(c)
        return h, (h,c)

### LSTM 2

In [10]:
class LSTMCell(Module):
    def __init__(self, ni, nh):
        self.ih = nn.Linear(ni,4*nh)
        self.hh = nn.Linear(nh,4*nh)

    def forward(self, input, state):
        h,c = state
        # One big multiplication for all the gates is better than 4 smaller ones
        gates = (self.ih(input) + self.hh(h)).chunk(4, 1)
        ingate,forgetgate,outgate = map(torch.sigmoid, gates[:3])
        cellgate = gates[3].tanh()

        c = (forgetgate*c) + (ingate*cellgate)
        h = outgate * c.tanh()
        return h, (h,c)

# Model 6

In [11]:
class LMModel6(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(res)
    
    def reset(self): 
        for h in self.h: h.zero_()

In [12]:
learn = Learner(dls, LMModel6(len(vocab), 64, 2), 
                loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=ModelResetter)
learn.fit_one_cycle(15, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.015898,2.699370,0.384196,00:01
1,2.129531,1.990293,0.290690,00:01
2,1.590543,1.769736,0.498617,00:01
3,1.301828,2.329215,0.476888,00:01
4,1.080614,2.432390,0.515951,00:02
5,0.864964,1.884870,0.606201,00:01
6,0.613487,1.557686,0.649984,00:01
7,0.380154,1.564096,0.666992,00:01
8,0.224778,1.320145,0.710612,00:01
9,0.134050,1.333302,0.726888,00:01


# Model 7

In [13]:
class Dropout(Module):
    def __init__(self, p): self.p = p
    def forward(self, x):
        if not self.training: return x
        mask = x.new(*x.shape).bernoulli_(1-p)
        return x * mask.div_(1-p)

In [14]:
class LMModel7(Module):
    def __init__(self, vocab_sz, n_hidden, n_layers, p):
        self.i_h = nn.Embedding(vocab_sz, n_hidden)
        self.rnn = nn.LSTM(n_hidden, n_hidden, n_layers, batch_first=True)
        self.drop = nn.Dropout(p)
        self.h_o = nn.Linear(n_hidden, vocab_sz)
        self.h_o.weight = self.i_h.weight
        self.h = [torch.zeros(n_layers, bs, n_hidden) for _ in range(2)]
        
    def forward(self, x):
        raw,h = self.rnn(self.i_h(x), self.h)
        out = self.drop(raw)
        self.h = [h_.detach() for h_ in h]
        return self.h_o(out),raw,out
    
    def reset(self): 
        for h in self.h: h.zero_()

In [15]:
learn_x = Learner(dls, LMModel7(len(vocab), 64, 2, 0.5),
                loss_func=CrossEntropyLossFlat(), metrics=accuracy,
                cbs=[ModelResetter, RNNRegularizer(alpha=2, beta=1)])

In [16]:
learn_x.fit_one_cycle(15, 1e-2, wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,2.703845,2.301995,0.435547,00:01
1,1.975745,1.857219,0.524821,00:01
2,1.307809,0.923792,0.724202,00:01
3,0.759294,0.828541,0.744873,00:01
4,0.455115,0.607791,0.822510,00:01
5,0.304398,0.467311,0.857341,00:01
6,0.227844,0.439848,0.865153,00:01
7,0.186000,0.391412,0.870036,00:01
8,0.160428,0.366598,0.879232,00:01
9,0.143744,0.340913,0.897542,00:01
